In [1]:
import pandas as pd
import lightningchart as lc

lc.set_license("LICENSE_KEY")

In [2]:
# Define the file path to the dataset containing global AQI values.
file_path_global = 'datasets/Global_AQI.csv'

# Read the CSV file into a Pandas DataFrame.
global_AQI = pd.read_csv(file_path_global)

# Display the first five rows of the DataFrame to preview the data.
global_AQI.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833


In [3]:
# Group the data by 'Country' and calculate the mean AQI value for each country.
country_aqi = global_AQI.groupby('Country')['AQI Value'].mean().reset_index()

# Display the first five rows of the resulting DataFrame to preview the country-level AQI data.
country_aqi.head()

,Country,AQI Value
0,Afghanistan,86.333333
1,Albania,77.111111
2,Algeria,106.250000
3,Andorra,32.000000
4,Angola,85.000000


In [4]:
import pycountry  # Used to get ISO 3166-1 alpha-3 country codes

# Extract all unique countries from the dataset.
unique_countries = country_aqi['Country'].unique()

# Create a dictionary to map country names to their ISO alpha-3 codes.
country_to_iso = {}
for country in unique_countries:
    try:
        # Find the ISO code for the country.
        country_data = pycountry.countries.lookup(country)
        country_to_iso[country] = country_data.alpha_3  # Map to alpha-3 ISO code (e.g., FIN for Finland)
    except LookupError:
        print(f"ISO code not found for: {country}")

# Manual mappings for countries that were not found automatically.
manual_ISO = {
    "Bolivia (Plurinational State of)": "BOL",
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Republic of Korea": "KOR",
    "State of Palestine": "PSE",
    "Turkey": "TUR",  # Needed because Turkey is listed as Türkiye in pycountry's DB
    "Venezuela (Bolivarian Republic of)": "VEN"
}

# Update the main dictionary with the manual ISO codes.
country_to_iso.update(manual_ISO)

# Map the ISO codes to the country_aqi DataFrame.
country_aqi['ISO_A3'] = country_aqi['Country'].map(country_to_iso)

# Filter out rows where the ISO code could not be found.
country_aqi_filtered = country_aqi.dropna(subset=['ISO_A3'])

# Prepare the final list of dictionaries with ISO codes and AQI values.
region_data = country_aqi_filtered[['ISO_A3', 'AQI Value']].rename(columns={'AQI Value': 'value'}).to_dict(orient='records')

# Display the first five entries of the final region data.
region_data[:5]

ISO code not found for: Bolivia (Plurinational State of)
ISO code not found for: Democratic Republic of the Congo
ISO code not found for: Iran (Islamic Republic of)
ISO code not found for: Republic of Korea
ISO code not found for: State of Palestine
ISO code not found for: Turkey
ISO code not found for: Venezuela (Bolivarian Republic of)


[{'ISO_A3': 'AFG', 'value': 86.33333333333333},
 {'ISO_A3': 'ALB', 'value': 77.11111111111111},
 {'ISO_A3': 'DZA', 'value': 106.25},
 {'ISO_A3': 'AND', 'value': 32.0},
 {'ISO_A3': 'AGO', 'value': 85.0}]

**Look up table values are from https://www.airnow.gov/aqi/aqi-basics/**

In [5]:
# Initialize a map chart of the world with a light theme.
chart = lc.MapChart(map_type='World', theme=lc.Themes.Light)

# Apply the AQI data to the map regions.
chart.invalidate_region_values(region_data)

# Set the color palette for the map based on AQI values.
chart.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('#00FF00')},    # Green for Good (0-50)
        {'value': 50, 'color': lc.Color('#00FF00')},
        {'value': 51, 'color': lc.Color('#FFFF00')},   # Yellow for Moderate (51-100)
        {'value': 100, 'color': lc.Color('#FFFF00')},
        {'value': 101, 'color': lc.Color('#FFA500')},  # Orange for Unhealthy for Sensitive Groups (101-150)
        {'value': 150, 'color': lc.Color('#FFA500')},
        {'value': 151, 'color': lc.Color('#FF0000')},  # Red for Unhealthy (151-200)
        {'value': 200, 'color': lc.Color('#FF0000')},
        {'value': 201, 'color': lc.Color('#800080')},  # Purple for Very Unhealthy (201-300)
        {'value': 300, 'color': lc.Color('#800080')},
        {'value': 301, 'color': lc.Color('#800000')},  # Maroon for Hazardous (301 and higher)
    ],
    look_up_property='value',
    percentage_values=False
)

# Open the map chart with live updates enabled.
chart.open(live=True)